<strong>Date :</strong> Créé le 03 Avril 2021| Mis à jour le 07 Avril 2021 </strong>

<strong>Compétition Kaggle - Team Théo
    
@auteur : </strong>Théo SACCAREAU

<strong>(2-3)_features_network1
      
Description :</strong> Le but de ce Notebook est de créer un premier réseau à partir de nos données Reddit et d'en déduire des features qui seront utilisées dans notre modèle de Machine Learning. Ce réseau aura pour noeuds soit des topics (sujets/posts auxquelles font références les commentaires => de type "t3") soit des commentaires (de type "t1"). Les liens seront des relations de parentés. 


Temps d'exécution du Notebook : environ  5/10min.

# Importation des librairies

In [1]:
# Librairies générales
import pandas as pd
import numpy as np
from tqdm import tqdm 

# Librairie pour les réseaux
import networkx as nx 

# Chemin 

In [2]:
# Chemin relatif vers le dossier "data" (inutile de le changer).
pathFile = "../data/" 

# Chargement des données d'entrée



In [3]:
# Chargement du fichier contenant le DataFrame retourné par le Notebook précédent.
# Temps d'exécution : 1min45.
df = pd.read_json(pathFile + "df_features_text.json")

In [4]:
df.head()

,created_utc,ups,link_id,name,author,parent_id,is_parent_comment,is_author_deleted,is_body_deleted,bot_comment,length_comment_chars_before_NLP,length_comment_chars_after_NLP,length_comment_words,nb_stopwords,sentiment,sentiment_child
0,1430438400,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,1,0,0,0,119,59,8,14,neg,0
1,1430438400,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,0,0,0,0,48,13,2,7,neg,0
2,1430438400,5.0,t3_34ffo5,t1_cqug90z,InterimFatGuy,t1_cqu80zb,1,0,0,0,4,4,1,0,neu,0
3,1430438401,1.0,t3_34aqsn,t1_cqug91c,JuanTutrego,t1_cqtdj4m,1,0,0,0,54,15,3,12,neg,0
4,1430438401,101.0,t3_34f9rh,t1_cqug91e,dcblackbelt,t1_cquc4rc,1,0,0,0,241,164,24,20,pos,0


# Création du réseau n°1 
Les noeuds sont soit des topics (sujets/posts auxquelles font références les commentaires, de type "t3") soit des commentaires (de type "t1"). <br> 
Les liens sont des relations de parentés. 


In [5]:
# Création d'un réseau dirigé vide
g = nx.DiGraph()

## Ajout des noeuds 

Nous commençons d'abord par la création des noeuds `topic` (ceux de la colonne `link_id`) et nous ajoutons un attribut de type `topic` à ce noeud.

In [6]:
g.add_nodes_from(df['link_id'], type="topic")

Nous continuons par la création des noeuds `commentaire` (ceux de la colonne `name`) et nous ajoutons un attribut de type `comment` à ce noeud. 

In [7]:
g.add_nodes_from(df['name'], type="comment")

## Ajout des arêtes 

L'idée ici est de considérer chaque paire (`name`, `parent_id`) dans le DataFrame et de créer une arête par paire. Ceci est fait en utilisant la fonction `add_edges_from` et en spécifiant `df[["name", "parent_id"]].values` comme liste d'arêtes (source, cible) à ajouter. De plus, nous créons un attribut `link_type` et l'instancions comme `parent` pour chaque arête.

In [8]:
g.add_edges_from(df[["name","parent_id"]].values, link_type="parent")

## Caractéristiques du graphe

In [9]:
print("Nombre de noeuds : ", len(g.nodes))
print("Nombre d'arêtes : ", len(g.edges))

Nombre de noeuds :  4396503
Nombre d'arêtes :  4234970


Les chiffres sont cohérents puisque ce réseau doit contenir autant de composantes connexes que de sujets (topics). Or, une composante connexe possèdent forcément n noeuds et n-1 arêtes. C'est donc logique que le nombre de noeuds soit légèrement supérieur au nombre d'arêtes. 


# Création de features 

## (1) Profondeur du commentaire 
L'idée de cette feature est de savoir à quelle profondeur le commentaire se trouve, c'est-à-dire, est-ce qu'il répond directement à un topic ou est-ce qu'il répond à un sous-commentaire, lui même sous-commentaire d'un commentaire d'un topic. <br> 
Selon nous, <strong>plus la profondeur du commentaire est grande, plus le commentaire est difficilement accessible, donc plus son score ne sera pas élevé (car moins d'utilisateurs le lieront). </strong>

In [10]:
# Pour chaque noeud du graphe, on regarde la plus grande longueur des plus
# courts chemins ayant pour début le noeud en cours. 
# Cela correspond alors à sa profondeur dans la discution (ce réseau étant 
# semblable à un "arbre").   
depth = [(node, max([len(path) for path in nx.shortest_path(
    g, source=node).values()])) for node in tqdm(g.nodes)]

100%|██████████| 4396503/4396503 [00:53<00:00, 81736.54it/s]


In [11]:
np.mean(np.array(np.array(depth)[:, 1], dtype=int))

3.7832918571874057

La profondeur moyenne des commentaires est de 3.78.

Remarque : cette moyenne prend en compte les noeuds qui sont des topics (t3) alors que dans notre DataFrame nous avons que des commentaires (t1), on suppose donc que la moyenne sera légèrement plus élevée (étant donné que la profondeur d'un topic est forcément de 1). 

In [12]:
# Dictionnaire qui met en relation un noeud et sa profondeur 
dict_comment_depth = dict(depth)

# Grâce au dictionnaire, on parcourt le DataFrame en attribuant une profondeur 
# à chaque noeud commentaire
df['depth'] = [dict_comment_depth[comment] for comment in tqdm(df['name'])]

100%|██████████| 4234970/4234970 [00:03<00:00, 1369059.92it/s]


In [13]:
np.mean(df['depth'])

3.889453998493496

Comme annoncé au-dessus, la moyenne est légèrement plus élevée si l'on ne prend en compte que les commentaires.

## (2) Nombre de réponses reçues 
Ici l'objectif de cette feature est de connaitre le nombre de réponses reçues par un commentaire (directement ou au total, c'est-à-dire, en comptant les sous-réponses). <br> 
Pour nous, <strong> plus un commentaire a fait réagir, plus il a été vu, et potentiellement, plus son score peut-être élevé</strong>.

### 2-1 Directement

In [14]:
# Pour connaitre le nombre de réponses directes d'un commentaire, 
# il faut calculer son degré entrant. 
nb_direct_resp = [(node, val) for (node, val) in tqdm(g.in_degree())]

100%|██████████| 4396503/4396503 [00:04<00:00, 1002986.73it/s]


In [15]:
max(np.array(np.array(nb_direct_resp)[:,1], dtype=int))

30771

Le commentaire/topic ayant reçu le plus de réponses directes en a reçues 30771. 

<br>

On se doute que c'est un topic qui a reçu autant de réponses directes, vérifions-le : 

In [16]:
[elem for elem in nb_direct_resp if elem[1] > 30000]

[('t3_37pr7d', 30771)]

In [17]:
[elem for elem in nb_direct_resp if elem[1] > 400 and 't1_' in elem[0]]

[('t1_cquvf6t', 430),
 ('t1_cr01zot', 456),
 ('t1_cr7jn0j', 519),
 ('t1_crh4y2b', 884),
 ('t1_crkjjl8', 681),
 ('t1_crlddy3', 439),
 ('t1_crnj9od', 416)]

C'est donc bel est bien un topic qui a reçu autant de réponses directes. Les commentaires ayant reçus le plus de réponses, en ont reçues environ 500. Cela semble être une nouvelle fois cohérent. 

In [18]:
pd.Series(np.array(np.array(nb_direct_resp)[:,1])).value_counts()

0       2931945
1       1037296
2        184807
3         67697
4         38018
         ...   
663           1
2993          1
1005          1
475           1
1493          1
Length: 726, dtype: int64

Enfin, on constate que la grande majorité des commentaires (près de 3 millions) n'ont reçu aucune réponse. 

In [19]:
# Dictionnaire qui met en relation un commentaire/topic et son nombre de réponses directes
dict_comment_direct_resp = dict(nb_direct_resp)

# Grâce au dictionnaire, on parcourt le DataFrame en attribuant un nombre de
# réponses directes à chaque commentaire
df['nb_direct_resp'] = [dict_comment_direct_resp[comment] for comment in tqdm(df['name'])]

100%|██████████| 4234970/4234970 [00:03<00:00, 1388368.04it/s]


In [20]:
np.mean(df['nb_direct_resp'])

0.5927302908875387

En moyenne, un commentaire reçoit 0.59 réponses. Il y a une forte hétérogénéïté entre les commentaires puisque près de 3 millions n'en reçoivent pas alors que certains en reçoivent plus de 500. 

### 2-2 Au total

In [21]:
# Pour chaque noeud, on fait la somme des degrés entrants de l'ensemble de ces fils. 
# Cela nous permet de connaitre le nombre de réponse totale. 

nb_total_resp = [(node, sum([g.in_degree[child] for child in set(
    [elem[0] for elem in nx.shortest_path(g, target=node).values()])])) for node in tqdm(g.nodes)]

100%|██████████| 4396503/4396503 [02:17<00:00, 31879.53it/s]


In [22]:
# On n'a plus besoin du réseau g, on libère de la RAM
del g

In [23]:
max(np.array(np.array(nb_total_resp)[:,1], dtype=int))

35812

Le commentaire/topic ayant reçu le plus de réponses au total, en a reçu 35812.
C'est exactement le chiffre que nous avions observé dans le Notebook sur l'observation et le nettoyage des données. <br> 
Ainsi, avec deux méthodes différentes, nous arrivons à la même conclusion, ce qui "valide" que ce réseau nous apporte bel et bien les informations souhaitées. 


In [24]:
# Dictionnaire qui met en relation un commentaire/topic et son nombre de réponses totales.  
dict_comment_total_resp = dict(nb_total_resp)

# Grâce au dictionnaire, on parcourt le DataFrame en attribuant un nombre de
# réponses directes à chaque commentaire
df['nb_total_resp'] = [dict_comment_total_resp[comment] for comment in tqdm(df['name'])]

100%|██████████| 4234970/4234970 [00:03<00:00, 1228762.20it/s]


In [25]:
np.mean(df['nb_total_resp'])

1.8894539984934957

En moyenne, un commentaire a reçu 1.89 réponses "totales" (en incluant les sous-réponses).

# Sauvegarde

In [26]:
# Sauvegarde DataFrame dans un fichier 
df.to_json(pathFile + "df_features_network1.json")